In [ ]:
from math import log, e, sqrt, pi

from scipy import stats
import tushare as ts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

sns.set_style('darkgrid') #seaborn不再默认修改，需要用set_style手动设置修改
mpl.rcParams['figure.figsize'] = (12,6)    # 设置matplotlib全局参数，指定所有绘图的大小

# 读入期权交易数据
data = pd.read_csv('data/option.csv')
data

#根据期权调整标记筛选数据
def get_modify_mark(ticker: str):
    """ 定义函数，从ticker字符串中获取期权合约调整标记 """
    return ticker[-6]

ticker = '510050C1503M02200'
get_modify_mark(ticker)

modifiy_mark = data.ticker.map(get_modify_mark) #提取所有期权数据的调整标记
modifiy_mark.head()

# 筛选并保留原始期权数据中调整标记为M的期权数据，即不考虑基础资产分红等情况
data = data[modifiy_mark == 'M'].copy()

#获取期权类型和行权价
def get_opt_type(ticker: str):
    """ 定义函数，从ticker字符串中获取期权类型 """
    return ticker[6].lower()

def get_strick(ticker: str):
    """ 定义函数，从ticker字符串中提取出期权的行权价格 """
    return int(ticker[-5:]) / 1000

# 从合约代码中获取合约行权价格
data['strikePrice'] = data.ticker.map(get_strick)

# 从合约代码中获取合约类型
data['optType'] = data.ticker.map(get_opt_type)

#计算期权剩余有效期
# 将字符串日期转换为日期时间类型
data['tradeDate'] = pd.to_datetime(data.tradeDate)

#计算到期日，将期权groupby，每组取最后一个交易日期的数据(一般是到期日，如果期权还未到期则是最后一个交易日的日期)
last_trade_date = data.groupby('ticker').tradeDate.last()
last_trade_date

# 将合约到期日期填入 data 数据表中
data['lastTradeDate'] = data.ticker.map(last_trade_date) #如果map中应用的function不是函数而是dataframe，则会进行对指定列进行广播并去掉该列保留其他列
data

# 计算合约剩余时间
data['daysLeft'] = (data.lastTradeDate - data.tradeDate).map(lambda x: x.days)
data

#计算期权匹配字符串
def get_match_mark(ticker: str): #包含了到期时间和行权价格
    return ticker[-10:]

# 根据合约代码计算match_mark，match_mark的作用是筛选出相同到期时间的认沽和认购期权
data['match_mark'] = data.ticker.map(get_match_mark)
data.head()

#获取50ETF行情数据
# 获取50etf价格数据
etf = ts.get_k_data('510050', '2014-01-01')
# 将日期列转换为日期时间格式，并设为索引，为数据拼接做准备
etf['date'] = pd.to_datetime(etf.date)
etf.set_index('date', inplace=True)
# 计算50etf日收益率
etf['d_return'] = etf.close.pct_change()
# 计算50etf的历史波动率
etf['volatility'] = etf.d_return.rolling(120).std() * sqrt(252)
etf

#计算隐含波动率
# 筛选特定到期时间的合约
days_left_selected = data[data.daysLeft == 30].copy()
days_left_selected

# 将50etf数据填入到筛选后的数据表
days_left_selected['etfClose'] = days_left_selected.tradeDate.map(etf.close)
days_left_selected['volatility'] = days_left_selected.tradeDate.map(etf.volatility)
days_left_selected

#牛顿法计算隐含波动率
def get_option_price(S, X, r, q, sigma, T, opt_type='call'):
    """
    根据BSM模型计算期权价格
    param S: 标的资产当前价格
    param X: 期权行权价格
    param r: 无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    param opt_type: 'call'或'put'
    return 返回期权价格
    """
    d1 = (log(S/X) + (r + sigma**2/2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    N1, N2 = stats.norm.cdf([d1, d2])
    call = S * N1 - X * e ** (-r * T) * N2      # 计算认购期权价格
             
    if opt_type.lower()[0] == 'c':
        return call
    else: 
        put = call + X * e ** (-r * T) - S  # 利用期权平价模型计算put价格
        return put

def vega(S, X, r, q, sigma, T):
    """
    定义函数计算期权的Vega
    param S: 标的资产当前价格
    param X: 期权行权价格
    param r: 复合无风险收益率
    param q: 基础资产分红收益率
    param sigma：标的资产年化波动率
    param T: 以年计算的期权到期时间
    return：返回期权的 Vega 值
    """
    d1 = (log(e**(-q*T)*S/X) + (r + sigma**2/2) * T) / (sigma * sqrt(T))
#     N_dash = e ** (-d1 ** 2 / 2)/sqrt(2 * pi)
    N_dash = stats.norm.pdf(d1)
    return S * N_dash * sqrt(T)

S = 2.676
X = 2.2
r = 0.0246
q = 0
sigma = 0.4
# T = 30/365
T = 0.08219

vega(S, X, r, q, sigma, T)

# 牛顿法计算隐含波动率
def get_implied_vol(S, X, r, q, T, V_mkt, opt_type='call'):
    """
    根据期权数据计算隐含波动率
    param S: 标的资产当前价格
    param X: 期权行权价格
    param r: 复合无风险收益率
    param q: 基础资产分红收益率
    param T: 以年计算的期权到期时间
    param V_mkt: 期权的市场价格
    param opt_type：期权类型，put或call
    return：返回隐含波动率
    """
    sigma = 0.5
    error = 10**(-6)
    
    for i in range(100):
        V = get_option_price(S, X, r, q, sigma, T, opt_type)      # BSM计算的期权价格
        if abs(V - V_mkt) < error:                                # 精度达到要求
            return sigma
        Vega = vega(S, X, r, q, sigma, T)  
        if Vega < 0.0001:                   # Vega太小的情况下会导致计算不收敛(对于ITM和OTM，当波动率很小时，Vega也会很小，导致下面式子的分母太小)，因此在Vega过小时直接返回一个很小的隐含波动率
            return np.nan
        sigma = sigma - (V - V_mkt)/ Vega   # 迭代计算sigma，g(x)=(V-V_mkt), g'(x)=vega
    return sigma

def get_iv_wrapper(option_record): #该函数用于提取series形式的参数(读取从DataFrame中切片的某行期权数据并计算为隐含波动率)
    """ 定义函数，根据期权数据信息计算对应的隐含波动率 """
    S = option_record.etfClose
    X = option_record.strikePrice
    r = 0.0246
    q = 0
    T = option_record.daysLeft / 365
    V_mkt = option_record.closePrice
    opt_type = option_record.optType
    return get_implied_vol(S, X, r, q, T, V_mkt, opt_type)

# 计算合约对应的隐含波动率
days_left_selected['impliedVol'] = days_left_selected.apply(get_iv_wrapper, axis=1)
days_left_selected

#根据隐含波动率筛选可以交易的期权对
# 仅保留隐含波动率不为空值的期权合约做下一步处理
iv_selcted = days_left_selected[days_left_selected.impliedVol.notna()].copy()
iv_selcted

# 将认沽和认购合约分开，为计算差值做准备
call = iv_selcted[iv_selcted.optType == 'c'].set_index(['match_mark', 'tradeDate'])
put = iv_selcted[iv_selcted.optType == 'p'].set_index(['match_mark', 'tradeDate'])

# 计算认沽和认购合约的隐含波动率差值
iv_diff = call.impliedVol - put.impliedVol
iv_diff

iv_diff.name = 'iv_diff' #改名，用于之后作为detaframe时的列名

# 对隐含波动率作图，观察认沽和认购期权隐含波动率差值的分布情况(左偏、长尾)，看图将波动率差的阈值定为-0.2，表示要做多call的隐含波动率，做空put的隐含波动率
iv_diff.hist(bins=50, alpha=0.5)
plt.show()

# 筛选出隐含波动率只差大于0.1的期权对
selected = pd.DataFrame(iv_diff[iv_diff < -0.2])
selected

#根据筛选结果计算每次交易的盈亏
#处理单一期权组合
# 获取期权对代码和起始交易时间
match_mark_sample = selected.index[1][0]
start_date_sample = selected.index[1][1]
start_date_sample

match_mark_sample

# 获取期权到期时间
data_selected = data[(data.match_mark == match_mark_sample) & (data.tradeDate >= start_date_sample)]
end_date_sample = data_selected.tradeDate.sort_values().iloc[-1]
end_date_sample

columns = ['start_date', 'end_date']

# 获得起始和终止时间的认购期权的收盘价
call = data_selected[(data_selected.optType == 'c')]
call_close = call.closePrice.iloc[[0, -1]]
call_close

# 获得起始和终止时间的认沽期权的收盘价
put = data_selected[(data_selected.optType == 'p')]
put_close = put.closePrice.iloc[[0, -1]]
put_close

# 获取起始和终止时间50etf的收盘价
etf_close = etf.close.loc[[start_date_sample, end_date_sample]]
etf_close

# 单笔交易的价格变动和盈亏情况
portfolio = pd.DataFrame([call_close.values, put_close.values, etf_close.values], index=['call', 'put', 'etf'], columns=['start', 'end'])
portfolio['factor'] = [1, -1, -1]
portfolio['profit'] = (portfolio.end - portfolio.start) * portfolio.factor
portfolio

# 计算交易盈亏
profit = portfolio.profit.sum()
profit

# 计算初始资金
initial_cost = portfolio.at['call', 'start'] + portfolio.at['etf', 'start'] * 0.5 - portfolio.at['put', 'start']    # 按照50%的做空保证金计算初始占用资金的情况
initial_cost

# 计算收益率
rate_of_return = profit / initial_cost
rate_of_return

#计算所有筛选出的期权对的交易结果
# 定义函数，获得一个期权组合对应的起止价格和50_etf的价格
def get_close_data(iv_diff_series, data_df, etf_df):
    match_mark = iv_diff_series.name[0]
    start_date = iv_diff_series.name[1]
    data_selected = data_df[(data_df.match_mark == match_mark) & (data_df.tradeDate >= start_date)]
    end_date = data_selected.tradeDate.sort_values().iloc[-1]
    call = data_selected[(data_selected.optType == 'c')]
    call_close = call.closePrice.iloc[[0, -1]]  
    put = data_selected[(data_selected.optType == 'p')]
    put_close = put.closePrice.iloc[[0, -1]]
    etf_close = etf.close.loc[[start_date, end_date]]
    result = pd.concat([call_close, put_close, etf_close], ignore_index=True)
    result.name =iv_diff_series.name 
    return result

# 对所有期权组合应用get_close_data()函数，获得相关的价格数据
# selected = pd.DataFrame(selected)
cost_profit = selected.apply(get_close_data, args=[data, etf], axis=1)
cost_profit.columns = ['call_start', 'call_end', 'put_start', 'put_end', 'etf_start', 'etf_end']
data_concat = pd.concat([selected, cost_profit], axis=1)
data_concat

# 计算收益
data_concat['call_pl'] = data_concat.call_end - data_concat.call_start
data_concat['put_pl'] = data_concat.put_start - data_concat.put_end
data_concat['etf_pl'] = data_concat.etf_start - data_concat.etf_end
data_concat['total_pl'] = data_concat.call_pl + data_concat.put_pl + data_concat.etf_pl
data_concat['initial_cost'] = data_concat.call_start - data_concat.put_start + data_concat.etf_start * 0.5     # 按照50%融券保证金比例计算
data_concat['rate_of_return'] = data_concat.total_pl / data_concat.initial_cost
data_concat

data_concat.rate_of_return.hist(bins=100, alpha=0.5)
plt.show()

data_concat[data_concat.rate_of_return == data_concat.rate_of_return.min()] #看收益率图发现有笔交易亏损很大，单独筛选出来看